In [3]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
import sys

In [7]:
src = cv2.imread('building.jpg', cv2.IMREAD_GRAYSCALE)

if src is None:
    print('Image load failed!')
    sys.exit()

tm = cv2.TickMeter()#연산 시간 측정 객체

# GFTT
tm.start()#시간 측정 시작

corners = cv2.goodFeaturesToTrack(src, 400, 0.01, 10)#(이미지, 코너검출최대수, 코너결정상수, 코너사이최소거리)

tm.stop()#시간 측정 종료
print('GFTT: {}ms.'.format(tm.getTimeMilli()))

dst1 = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

if corners is not None:
    for i in range(corners.shape[0]):
        pt = (int(corners[i, 0, 0]), int(corners[i, 0, 1]))
        cv2.circle(dst1, pt, 5, (0, 0, 255), 2)

# FAST
tm.reset()
tm.start()

fast = cv2.FastFeatureDetector_create(60)#(임계값)
keypoints = fast.detect(src)

tm.stop()
print('FAST: {}ms.'.format(tm.getTimeMilli()))

dst2 = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

for kp in keypoints:
    pt = (int(kp.pt[0]), int(kp.pt[1]))
    cv2.circle(dst2, pt, 5, (0, 0, 255), 2)

cv2.imshow('src', src)
cv2.imshow('dst1', dst1)
cv2.imshow('dst2', dst2)
cv2.waitKey()

cv2.destroyAllWindows()


GFTT: 56.7315ms.
FAST: 5.4139ms.


2.이미지 피라미드\
이미지를 확대. 축소 샘플링(4배, 1/4배 등)\
동작: upsize(zoom in ) / downsize(zoom out)\



[guassian pyramid]\
1) Gaussian Pyramid:이미지다운 샘플링. 모든 레이어는 아래에서 위로 번호가 매겨짐 (i+1). 짝수번째 행과 열을 제거함으로 축소 (1, 1/4, 1/8)\
2) Laplacian Pyramid: 위 가우시안 피라미드 아래층의 이미지로 업 샘플링된 이미지 재구성(해상도 낮음)\



<Gaussian Pyramid 예>\
ig = cv2.imread('k.jpg')\
lower_reso = cv2.pyrDown(img) #원본 이미지 1/4 사이즈\
higher_reso = cv2.pyrUp(img)원본이미지 4배 사이즈\


<Laplacian Pyramid 예>\
가우시안 피라미드로 이미지를 축소했다가 다시 확대하면 이 과정에서 행이나 열이 소실된다. 이렇게 생성된 이미지를 원본 이미지 크기로 변경한 뒤 원본 이미지에서 피라미드 이미지의 차 를 계산하면 이미지 운곽 도출

In [11]:
import cv2 

img = cv2.imread('building.jpg')

s = cv2.pyrDown(img)  # 원본 이미지의 1/4사이즈
l = cv2.pyrUp(s)  #원본 이미지의 4배 사이즈\

h,w,c = img.shape
l2 = cv2.resize(l,(w,h))
res = cv2.subtract(img,l2)

cv2.imshow('img',res)
cv2.waitKey(0)
cv2.destroyAllWindows()

5. 특징점 매칭\
특징점 매칭(feature point matching)\
- 두 영상에서 추출한 특징점 기술자를 비교하여 서로 유사한 기술자를 찾는 작업\
특징 벡터 유사도 측정 방법\
- 실수 특징 벡터: L2 노름(L2 norm)사용\
- 이진 특징 벡터: 해밍 거리 (hamming distance)사용\
